Robot Dynamic Model Identification
=======
---

# 1 - Praparing work

## Import libraries

In [1]:
# enable auto-reload modules
%load_ext autoreload
%autoreload 2
#interactive plotting in separate window
%matplotlib qt 
#%matplotlib inline

from sympy import init_printing
init_printing()
    
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]

import numpy as np
import sympy
from robot_def import RobotDef
from kinematics import Geometry
from dynamics import Dynamics
from trajectory_optimization import TrajOptimizer
from trajectory_optimization import TrajPlotter
from utils import new_sym
from utils import save_data, load_data
import time

## Folder dir for saving and loading files

In [2]:
# model_name = 'test_psm_long2'
# model_name ='test'
# model_name = 'mtm_3links'
model_name = 'mtm'
# model_name = 'mtm_3links_parallel'
# model_name = 'mtm_4links_parallel'

model_folder = 'data/' + model_name + '/model/'

---
# 2 - Robot modelling

## Create joint variables and define their relations

## Robot geometry definition in following order

| Joint number | prev link | succ links | $a$ | $\alpha$ | $d$ | $\theta$ | link inertia | motor inertia | friction |
|--------------|-----------|------------|-----|----------|-----|----------|--------------|---------------|---------|

In [8]:
q0, q1, q2, q3, q4, q5, q6, q7, q8, q9, q10 = new_sym('q:11')
_pi = sympy.pi

dh = []
springs = []
friction_type = []

if model_name =='test':
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, False),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, False)]
    q = q1
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.067
    q_o = 23.0/180*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s]
    friction_type = ['Coulomb', 'viscous', 'offset']
elif model_name == 'mtm':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.067
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    #springs = [tau_s]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print("MTM")
elif model_name == 'mtm_3links_parallel':
    # define paralelogram coordinate relation
    q3 = -q2 + q4
    q5 = -q4 + q2
    q6 = q4 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,4],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   1,  [5],    0,     -_pi/2, 0,        q4+_pi,   True,  False, True),
          (5,   4,  [6],   0.1,   0,      0,        q5-_pi/2, True,  False, True),
          (6,   5,  [],     0.279, 0,      0,        q6,      False, False, True)]
    friction_type = ['Coulomb', 'viscous']
    #springs = [q1, q2, q3]
elif model_name == 'mtm_4links_parallel':
    # define paralelogram coordinate relation
    q3 = -q2 + q5
    q6 = -q5 + q2
    q7 = q5 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,5],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [0],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   1,  [6],     0,     -_pi/2, 0,        q5+_pi,   True,  False, True),
          (6,   5,  [7],   0.1,   0,      0,        q6-_pi/2, True,  False, True),
          (7,   6,  [],     0.279, 0,      0,        q7,      False, False, True)]
    friction_type = ['Coulomb', 'viscous']
elif model_name == 'mtm_3links':
    dh = [(0,   -1, [1],    0,      0,      0,          0,         False, False, False),
          (1,   0,  [2],    0,      0,      -0.21537,   q1,        True,  False, True),
          (2,   1,  [3],    0,      -_pi/2, 0,          q2+_pi/2,  True,  False, True),
          (3,   2,  [4],    0.279,  0,      0,          q3+_pi/2,  True,  True,  True)]
    # springs = [q1]
    friction_type = ['Coulomb', 'viscous']
elif (model_name == 'test_psm'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    x = [None]*10
    x[0] = (0,   -1, [1],        0,              0,     0,                       0,      False,  False, False) # Base
    x[1] = (1,   0,  [2],        0,     sympy.pi/2,     0,           q1+sympy.pi/2,       True,  False,  True) # Yaw
    x[2] = (2,   1,  [3, 4],        0, -sympy.pi/2,     0,              sympy.pi/2,       False, False, False) # -- Intermediate
    x[3] = (3,   2,  [5, 6],  -0.0296,           0,     0,    q2-beta - sympy.pi/2,        True,  False, True) # Pitch Back
    x[4] = (4,   2,  [] ,     0.0664 ,           0,     0,    q2-beta - sympy.pi/2,        True,  False, True) # Pitch Front                                    )
    x[5] = (5,   2,  [7],     -0.0296,           0,     0,   q2-beta2 - sympy.pi/2,        False,  False, False)    # Intermediate
    x[6] = (6,   3,  [8],       0.150,           0,     0,    -q2+beta+sympy.pi/2,      True,  False,  True)    # Pitch Bottom
    x[7] = (7,   5,  [],       0.1842,           0,     0,    -q2+beta2+sympy.pi/2,     True,  False,  True)    # Pitch Top
    x[8] = (8,   6,  [9],       0.516,           0,     0,                      q2,     True,  False,  True)    # Pitch End
    x[9] = (9,   8,  [10],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,     True,  False, True)    # Main Insertion
    dh = x
    friction_type = ['Coulomb', 'viscous', 'offset']              
elif (model_name == 'test_psm_long2'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    
    x = [None]*17
    x[0]  = ( 0,  -1,  [1],          0,           0,     0,                       0, False, False, False) # Base
    x[1]  = ( 1,   0,  [2],          0,  sympy.pi/2,     0,           q1+sympy.pi/2,  True, False,  True) # Yaw
    x[2]  = ( 2,   1,  [3, 4, 5],       0, -sympy.pi/2,     0,              sympy.pi/2, False, False, False) # -- Intermediate
    x[3]  = ( 3,   2,  [6, 10], -0.0296,       0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Back
    x[4]  = ( 4,   2,  [] ,     0.0664,           0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Front                                    )
    x[5]  = ( 5,   2,  [7],    -0.0296,           0,     0,   q2-beta2 - sympy.pi/2, False, False, False)    # Intermediate
    x[6]  = ( 6,   3,  [8],      0.150,           0,     0,     -q2+beta+sympy.pi/2,  True, False,  True)    # Pitch Bottom
    x[7]  = ( 7,   5,   [],     0.1842,           0,     0,    -q2+beta2+sympy.pi/2,  True, False,  True)    # Pitch Top
    x[8]  = ( 8,   6,  [9],      0.516,           0,     0,                      q2,  True, False,  True)    # Pitch End
    x[9]  = ( 9,   8, [12],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,  True, False,  True)    # Main Insertion
    x[10] = (10,   3, [11],          0,           0,     0,         beta+sympy.pi/2, False, False, False) # Intermediate Counterweight
    x[11] = (11,  10,   [],       -0.1, +sympy.pi/2, delta*q3,                    0,  True, False,  True) # Counterweight
    
    x[12] = (12,   9, [13],          0,           0,0.4162,          -sympy.pi/2+q4, False,  True,  True) # Tool Roll 
    x[13] = (13,  12, [14],          0, -sympy.pi/2,     0,          -sympy.pi/2+q5, False,  True,  True) # Tool Pitch
    x[14] = (14,  13, [15,16],  0.0091, -sympy.pi/2,     0,          -sympy.pi/2+q6, False,  True,  True) # Tool Yaw
    x[15] = (15,  14, [],            0,           0,     0,                     -q7, False, False,  True) # Gripper1
    x[16] = (16,  15, [],            0,           0,     0,                      q7, False,  True,  True) # Gripper2
    dh = x
    friction_type = ['Coulomb', 'viscous', 'offset']
else:
    raise Exception("No robot name matches " + name)
print(friction_type)
robot_def = RobotDef(dh,
                     springs=springs,
                     dh_convention='mdh',
                     friction_type=friction_type)

MTM
['Coulomb', 'viscous', 'offset']


In [72]:
robot_def.bary_params

## Create kinematics chain

In [73]:
geom = Geometry(robot_def)

In [74]:
angle = [0, 0, 0, 0, 0, 0, 0]
geom.draw_geom(angle)

## Create dynamics

In [75]:
%time dyn = Dynamics(robot_def, geom)

Calculating Lagrangian...
Calculating the link kinetic energy of 1/10
Calculating the link kinetic energy of 2/10
Calculating the link kinetic energy of 3/10
Calculating the link kinetic energy of 4/10
Calculating the link kinetic energy of 5/10
Calculating the link kinetic energy of 6/10
Calculating the link kinetic energy of 7/10
Calculating the link kinetic energy of 8/10
Calculating the link kinetic energy of 9/10
Calculating joint torques...
tau of q1
tau of q2
tau of q8
tau of q4
tau of q5
tau of q6
tau of q7
Adding frictions, motor rotor inertia and springs...
Calculating gregressor...
('input_vars', (q1, q2, q8, q4, q5, q6, q7, dq1, dq2, dq8, dq4, dq5, dq6, dq7, ddq1, ddq2, ddq8, ddq4, ddq5, ddq6, ddq7))
Calculating M, C and G...
Calculating base parameter...
[ 79  82  80  66  69  67  81  53  83  85  84  72  56  71  78  54  59  58
  70  40  45  46  43  41  47  32  57 111 110  14  68  92  44  42  37  15
  17  30  93  16  55  97  98  18  50  29  76  63  89  11  24  38 102  31
  9

In [78]:
robot_def.bary_params

In [79]:
sympy.Matrix(dyn.base_param)

⎡                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                   

In [80]:
from data import RobotModel

# Data to be saved
robot_model = RobotModel(dyn)

# Save
save_data(model_folder, model_name, robot_model)
print('Saved {} parameters'.format(len(robot_model.base_param)))

('Yes DONE', 69)


---
# 3 - Excitation trajectory optimization

In [3]:
# model_name = 'test_psm_long2'
# model_name ='test'
# model_name = 'mtm_3links'
model_name = 'mtm'
#model_name = 'mtm_3links_parallel'
# model_name = 'mtm_4links_parallel'

Load Data

In [21]:
from utils import load_data
model_folder = 'data/' + model_name + '/model/'
robot_model = load_data(model_folder, model_name)

## Create trajectory optimizer object, with H matrix, foourier order, base frequency, joint constraints and cartisian constraints as input

In [22]:
#trajectory_name = 'one'
trajectory_name = 'two_normalized'

In [23]:
from numpy import deg2rad

optimal_trajectory_folder = 'data/optimal_trajectory/'
trajectory_folder = 'data/' + model_name +'/optimal_trajectory/'

base_freq = 0.1
fourier_order = 6

joint_constraints = []
cartesian_constraints = []

if model_name == 'mtm':
#     joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q4, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q5, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q6, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q7, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q8, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]
#     joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q4, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q5, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q8, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]

    joint_constraints = [(q1, deg2rad(-60), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q2, deg2rad(-12), deg2rad(62), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-30), deg2rad(30), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-90), deg2rad(195), -4*np.pi, 4*np.pi),
                         (q5, deg2rad(-87), deg2rad(180), -4*np.pi, 4*np.pi),
                         (q6, deg2rad(-40), deg2rad(38), -4*np.pi, 4*np.pi),
                         (q7, deg2rad(-460), deg2rad(450), -10*np.pi, 10*np.pi),
                         (q8, deg2rad(-9), deg2rad(39), deg2rad(-150), deg2rad(150))]

#     cartesian_constraints = [(3,'max',1,0.4,1),
#                              (3,'min',-1,-1,-1)]
elif model_name == 'mtm_3links':
    joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
                         (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
                         (q3, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]
    
    cartesian_constraints = [(1,'max',1,0.4,1),
                             (1,'min',-1,-1,-1),
                             (2,'max',1,0.4,1),
                             (2,'min',-1,-1,-1)]
    
    # MTM STUFF
#jnt_cnst = [(q1, -0.698132, 1.13446, -2, 2),(q2, -0.261799, 0.872665, -2, 2), (q3,-0.172665, 0.610865, -2, 2)]
#jnt_cnst = [(q1, -1.5, 1.5, -2, 2),(q2, -1.5, 1.5, -2, 2), (q3,-1.5, 1.5, -2, 2)]    

#crt_cnst = [(4, 'max', -0.283, 0.279, -0.25137), (4, 'min', -0.5, -0.321, -0.39137)]
#crt_cnst = [(3, 'max', -0.300, 0.3, -0.26137), (3, 'min', -0.5, -0.45, -0.53137)]
elif model_name == 'mtm_3links_parallel' or model_name == 'mtm_3links_parallel_offset':
    joint_constraints = [(q1, deg2rad(-60), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q2, deg2rad(-12), deg2rad(62), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-32), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-9), deg2rad(39), deg2rad(-360), deg2rad(360))]
elif model_name == 'mtm_4links_parallel':
    joint_constraints = [(q1, deg2rad(-60), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q2, deg2rad(-12), deg2rad(62), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-32), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-90), deg2rad(195), -2*np.pi, 2*np.pi),
                         (q5, deg2rad(-9), deg2rad(39), deg2rad(-360), deg2rad(360))]
elif model_name == 'psm':
    pass
elif model_name == 'test_psm_long2':
    joint_constraints = [(q1, -1.4, 1.4, -2, 2),
                         (q2, -0.5, 0.8, -2, 2),
                         (q3,0.05,0.23,-0.4,0.4),
                         (q4,-1.5,1.5,-0.4,0.4),
                         (q5,-1.5,1.5,-0.4,0.4), 
                         (q6,-1.5,1.5,-0.4,0.4),
                         (q7,0.05, 0.75,-0.4,0.4)]
else:
    raise Exception("No robot name matches " + model_name)
    
traj_optimizer = TrajOptimizer(robot_model, fourier_order, base_freq,
                               joint_constraints=joint_constraints,
                               cartesian_constraints = cartesian_constraints)
                                                                                                 
                                                                                                                     
traj_optimizer.optimize()


joint constraint number: 8
cartesian constraint number: 0
constraint number: 32
frames_constrained: []
cond: 108.284987025
cond: 108.284987025
cond: 108.284986938
cond: 108.284985001
cond: 108.284987998
cond: 108.284990253
cond: 108.284993406
cond: 108.284987715
cond: 108.284987717
cond: 108.284993776
cond: 108.284989857
cond: 108.284989895
cond: 108.284987681
cond: 108.284980976
cond: 108.284968686
cond: 108.284987159
cond: 108.284982106
cond: 108.284982243
cond: 108.284984375
cond: 108.284990565
cond: 108.28498608
cond: 108.285011665
cond: 108.285005277
cond: 108.285002504
cond: 108.285002372
cond: 108.28499592
cond: 108.284995183
cond: 108.284944628
cond: 108.284986461
cond: 108.284994185
cond: 108.284979144
cond: 108.284986077
cond: 108.284975508
cond: 108.28497328
cond: 108.285080737
cond: 108.285015893
cond: 108.285001938
cond: 108.28499235
cond: 108.28498416
cond: 108.28498242
cond: 108.284987301
cond: 108.284984453
cond: 108.284988037
cond: 108.284987959
cond: 108.284988316
con

cond: 213.619738592
cond: 213.619704766
cond: 213.619558749
cond: 213.619769601
cond: 213.619695749
cond: 213.619685393
cond: 213.61968714
cond: 213.619690356
cond: 213.619694864
cond: 213.619691491
cond: 213.619687433
cond: 213.619693051
cond: 213.619687733
cond: 213.619694465
cond: 213.619697578
cond: 213.619684569
cond: 213.619692959
cond: 213.619691662
cond: 213.619700345
cond: 213.619688793
cond: 213.619694059
cond: 213.619691802
cond: 213.619690354
cond: 213.619692923
cond: 213.619689606
cond: 213.619690095
cond: 213.619689584
cond: 213.619687748
cond: 213.61968896
cond: 213.619687339
cond: 213.619689825
cond: 213.619688836
cond: 213.619686192
cond: 213.619687486
cond: 213.619689792
cond: 213.619694997
cond: 213.619687674
cond: 213.619690437
cond: 213.61970938
cond: 213.619694085
cond: 213.619693484
cond: 213.619693864
cond: 213.619695007
cond: 213.619689561
cond: 213.619689982
cond: 213.619690036
cond: 213.61969013
cond: 213.619691511
cond: 213.619689398
cond: 213.619689812
cond

cond: 248.939197299
cond: 248.939194651
cond: 248.939196499
cond: 248.939193748
cond: 248.939196983
cond: 248.939196943
cond: 248.939197353
cond: 248.939197093
cond: 248.939196343
cond: 248.93919963
cond: 248.939198288
cond: 248.93919747
cond: 248.939196925
cond: 248.939196544
cond: 248.939196989
cond: 251.860663571
cond: 246.109436706
cond: 246.109436706
cond: 246.109429299
cond: 246.109442328
cond: 246.109421057
cond: 246.109434543
cond: 246.109431253
cond: 246.109448861
cond: 246.109415153
cond: 246.109433475
cond: 246.10943494
cond: 246.109425862
cond: 246.109437493
cond: 246.109386735
cond: 246.109401259
cond: 246.109537833
cond: 246.10932294
cond: 246.109368731
cond: 246.109346928
cond: 246.109479889
cond: 246.109508786
cond: 246.109269062
cond: 246.10947524
cond: 246.10940047
cond: 246.1094886
cond: 246.109335337
cond: 246.109443739
cond: 246.109453312
cond: 246.109074538
cond: 246.109607468
cond: 246.109426753
cond: 246.10947506
cond: 246.109338028
cond: 246.109327811
cond: 246

cond: 196.134092571
cond: 196.134106456
cond: 196.133859179
cond: 196.134176198
cond: 196.134071212
cond: 196.134063609
cond: 196.134024766
cond: 196.134051943
cond: 196.134193299
cond: 196.134088936
cond: 196.134023554
cond: 196.134063346
cond: 196.134114469
cond: 196.134069525
cond: 196.134080747
cond: 196.134073073
cond: 196.134072733
cond: 196.134068668
cond: 196.134073113
cond: 196.134073533
cond: 196.134075976
cond: 196.134087896
cond: 196.134076333
cond: 196.134070549
cond: 196.134074098
cond: 196.134075514
cond: 196.134080023
cond: 196.13407574
cond: 196.134080217
cond: 196.134080622
cond: 196.134077118
cond: 196.134077721
cond: 196.134074954
cond: 196.134071825
cond: 196.1340773
cond: 196.134077601
cond: 196.134076215
cond: 196.134078444
cond: 196.134077452
cond: 196.134077507
cond: 196.13407603
cond: 196.134077916
cond: 196.13407448
cond: 196.134077667
cond: 196.134073342
cond: 196.134075415
cond: 196.134075229
cond: 196.134079421
cond: 196.134079926
cond: 196.134076844
cond:

cond: 184.446066482
cond: 184.446065494
cond: 184.446065513
cond: 184.446064505
cond: 184.446062816
cond: 184.44606462
cond: 184.446064022
cond: 184.44606502
cond: 184.44606651
cond: 184.446066893
cond: 184.446060443
cond: 184.446064012
cond: 184.446065352
cond: 184.446067924
cond: 184.446065957
cond: 184.44606471
cond: 184.446065357
cond: 184.446065288
cond: 184.446065453
cond: 184.446065934
cond: 184.446065506
cond: 184.44606561
cond: 184.446065829
cond: 184.446065757
cond: 184.446064837
cond: 184.446065058
cond: 184.446065569
cond: 184.446065146
cond: 184.446065413
cond: 196.175444916
cond: 182.62772257
cond: 182.62772257
cond: 182.627723916
cond: 182.627722993
cond: 182.627710821
cond: 182.627734724
cond: 182.627702757
cond: 182.627728136
cond: 182.627700814
cond: 182.627720508
cond: 182.627702993
cond: 182.627708228
cond: 182.627725342
cond: 182.627671382
cond: 182.627712631
cond: 182.627759712
cond: 182.627595188
cond: 182.627720059
cond: 182.627683923
cond: 182.627760773
cond: 1

...

cond: 176.252374983
cond: 176.252375047
cond: 176.252375175
cond: 176.252375333
cond: 176.252375159
cond: 176.252375265
cond: 179.538381116
cond: 175.963632526
cond: 175.963632526
cond: 175.963632174
cond: 175.963633333
cond: 175.963621488
cond: 175.963642485
cond: 175.963606255
cond: 175.963637996
cond: 175.963607614
cond: 175.963628836
cond: 175.963614322
cond: 175.96362388
cond: 175.963636489
cond: 175.963584508
cond: 175.963630408
cond: 175.963664013
cond: 175.963502322
cond: 175.963628842
cond: 175.963594553
cond: 175.96367735
cond: 175.963688516
cond: 175.963384425
cond: 175.963620915
cond: 175.963643423
cond: 175.963633529
cond: 175.963586948
cond: 175.963637379
cond: 175.963666659
cond: 175.963420793
cond: 175.963740153
cond: 175.963597657
cond: 175.963635679
cond: 175.963579919
cond: 175.963605292
cond: 175.963801465
cond: 175.963642806
cond: 175.96359116
cond: 175.963631138
cond: 175.963671075
cond: 175.963639806
cond: 175.963631998
cond: 175.963632525
cond: 175.963632736
con

cond: 175.510151209
cond: 175.51036082
cond: 175.510191887
cond: 175.510140766
cond: 175.510179365
cond: 175.510230433
cond: 175.510189257
cond: 175.51018328
cond: 175.510183461
cond: 175.510183262
cond: 175.510183423
cond: 175.510183325
cond: 175.510183209
cond: 175.510183311
cond: 175.510183292
cond: 175.510183473
cond: 175.510183159
cond: 175.510183001
cond: 175.510183234
cond: 175.510183007
cond: 175.510183185
cond: 175.510182336
cond: 175.510182285
cond: 175.510182952
cond: 175.510182957
cond: 175.510182946
cond: 175.51018303
cond: 175.510183095
cond: 175.510182958
cond: 175.510183056
cond: 175.510183047
cond: 175.5101834
cond: 175.51018296
cond: 175.510183453
cond: 175.510183115
cond: 175.510183258
cond: 175.510182469
cond: 175.510183277
cond: 175.510182979
cond: 175.510183262
cond: 175.510183692
cond: 175.510182969
cond: 175.510182753
cond: 175.510182925
cond: 175.510183142
cond: 175.510183041
cond: 175.510183138
cond: 175.510183042
cond: 175.510183283
cond: 175.510183123
cond: 

cond: 175.13920413
cond: 175.139204841
cond: 175.139204474
cond: 175.139204681
cond: 175.139205082
cond: 175.13920412
cond: 175.139204287
cond: 175.139204746
cond: 175.139204289
cond: 175.139204579
cond: 175.139204537
cond: 175.139204126
cond: 175.139204667
cond: 175.139204407
cond: 175.139204546
cond: 175.139204535
cond: 175.139204522
cond: 175.139204274
cond: 175.139204393
cond: 175.139204592
cond: 175.139204645
cond: 175.139204594
cond: 175.139204567
cond: 177.581171619
cond: 175.118852544
cond: 175.118852544
cond: 175.118853376
cond: 175.118852482
cond: 175.118842077
cond: 175.118861261
cond: 175.118826151
cond: 175.118857738
cond: 175.118827609
cond: 175.118847343
cond: 175.118833354
cond: 175.1188441
cond: 175.118857355
cond: 175.118806339
cond: 175.118845397
cond: 175.118876812
cond: 175.118726639
cond: 175.118849752
cond: 175.118815818
cond: 175.118899438
cond: 175.118915116
cond: 175.118606168
cond: 175.118838063
cond: 175.118861875
cond: 175.118850656
cond: 175.118813829
cond

cond: 174.86429528
cond: 174.864261101
cond: 174.864344481
cond: 174.864359792
cond: 174.864052264
cond: 174.86428279
cond: 174.86430825
cond: 174.864296241
cond: 174.864259572
cond: 174.864302651
cond: 174.864328393
cond: 174.864099346
cond: 174.864409372
cond: 174.864263881
cond: 174.864302732
cond: 174.864243893
cond: 174.864260401
cond: 174.864478796
cond: 174.864307893
cond: 174.86425794
cond: 174.864296029
cond: 174.864336309
cond: 174.864305673
cond: 174.864298465
cond: 174.864299294
cond: 174.86429854
cond: 174.864298217
cond: 174.864298083
cond: 174.864298517
cond: 174.864297958
cond: 174.864297898
cond: 174.864298717
cond: 174.864298672
cond: 174.864297952
cond: 174.864297676
cond: 174.864298296
cond: 174.864298031
cond: 174.864297887
cond: 174.864298398
cond: 174.864298259
cond: 174.864298107
cond: 174.86429808
cond: 174.864298078
cond: 174.864298412
cond: 174.864298039
cond: 174.864298122
cond: 174.864298211
cond: 174.864298054
cond: 174.86429813
cond: 174.864297955
cond: 1

cond: 174.697552511
cond: 174.697552433
cond: 174.697552352
cond: 174.697552868
cond: 174.697552329
cond: 174.697552308
cond: 174.697552441
cond: 174.697552313
cond: 174.697552408
cond: 174.697552357
cond: 174.697551923
cond: 174.697552025
cond: 174.697552049
cond: 174.697552433
cond: 174.697552432
cond: 174.697552438
cond: 174.697552351
cond: 174.697552091
cond: 174.697552197
cond: 174.697552497
cond: 174.697552397
cond: 174.697552299
cond: 174.697552365
cond: 174.697552208
cond: 174.697552366
cond: 174.697552331
cond: 174.697552372
cond: 174.69755237
cond: 174.697552379
cond: 174.697552364
cond: 174.697552379
cond: 174.697552369
cond: 174.697552402
cond: 174.697552373
cond: 174.697552362
cond: 175.048711472
cond: 174.698599662
cond: 174.697563559
cond: 174.697526499
cond: 174.697542529
cond: 174.697548617
cond: 174.697550936
cond: 174.697551821
cond: 174.697552158
cond: 174.697552287
cond: 174.697552337
cond: 174.697552337
cond: 174.697552742
cond: 174.697552227
cond: 174.697541754
c

In [24]:
reg_norm_mat = traj_optimizer.calc_normalize_mat()

## Plot optimal excitation trajectory

In [25]:
traj_optimizer.calc_frame_traj()

In [27]:
traj_plotter = TrajPlotter(traj_optimizer.fourier_traj,traj_optimizer.frame_traj,
                           traj_optimizer.const_frame_ind, robot_model.coordinates)
traj_plotter.plot_desired_traj(traj_optimizer.x_result)

traj_plotter.plot_frame_traj(True)

## Save trajectory for later use

In [17]:
dof_order_bf_x_norm = (traj_optimizer.fourier_traj.dof, fourier_order,
                       base_freq, traj_optimizer.x_result, reg_norm_mat)
save_data(trajectory_folder, trajectory_name, dof_order_bf_x)

freq = 500
tf = 35 # including 5 seconds' prestable time
traj_optimizer.make_traj_csv(trajectory_folder, trajectory_name, freq, tf)

In [18]:
from IPython.display import HTML, display
import tabulate

table = []
table.append(["joint", 'qo'] +
             ["a"+str(i+1) for i in range(fourier_order)] +
             ["b"+str(i+1) for i in range(fourier_order)])
for i in range(traj_optimizer.fourier_traj.dof):
    line = []
    line.append(robot_model.coordinates[i])
    line += np.round(traj_optimizer.x_result[i*(1 + fourier_order*2): (i+1)*(1 + fourier_order*2)], 4).tolist()
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt='html')))

joint,qo,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6
q1,-0.2479,-0.0193,0.0017,0.1973,0.4971,0.6819,-0.5573,0.0259,0.1275,0.2107,0.0604,-0.4959,2.4405
q2,0.3162,0.2304,-0.2278,0.1893,-0.4222,0.0119,-0.0124,0.1002,-0.0062,0.4082,-0.3379,-0.6869,-0.4395
q8,0.2396,0.0367,0.0718,-0.0939,-0.3023,0.2761,0.6476,0.1186,-0.0955,0.3206,-0.032,-0.0335,0.0596
q4,1.1273,0.8791,-0.6371,-1.6396,0.0819,-1.3904,2.0802,-0.0347,-0.6973,-0.8018,-2.4749,0.9701,-1.1764
q5,0.9624,-0.2276,-0.6137,-0.2506,-0.7712,2.4053,0.1679,-0.3808,0.7127,0.6091,-2.3132,1.9517,0.381
q6,-0.2692,-0.0628,-0.0169,-0.2313,-0.0463,-0.1869,-0.035,0.0041,-0.0608,-0.0181,0.1218,-0.2472,-0.591
q7,1.1847,-0.2828,2.3554,1.1553,2.4601,4.1282,-0.2198,-0.0117,6.2194,-2.2045,4.7942,-1.5199,2.4431


---
# Data processing

In [85]:
from identification import load_trajectory_data, diff_and_filt_data, plot_trajectory_data, plot_meas_pred_tau, gen_regressor

## Load data

In [86]:
# Names as Arguments
# model_name = 'test_psm_long'
# model_name = 'mtm_3links'
model_name = 'mtm'
# model_name == 'mtm_3links_parallel'
# model_name == 'mtm_4links_parallel'

trajectory_name = 'one'
# trajectory_name = 'two'

In [87]:
# Folders Standard
model_folder = 'data/' + model_name + '/model/'
trajectory_folder = 'data/' + model_name +'/optimal_trajectory/'
results_folder = 'data/' + model_name +'/measured_trajectory/'
results_data_file = results_folder + trajectory_name + '_results.csv'

# Load Results
trajectory_sampling_rate = 500
#t, q_raw, tau_raw = load_trajectory_data(results_data_file, trajectory_sampling_rate)
t, q_raw, dq_raw, tau_raw = load_trajectory_data(results_data_file, trajectory_sampling_rate)
dof = q_raw.shape[1]

# Load Robot Model
robot_model = load_data(model_folder,model_name)

(<type 'numpy.ndarray'>, (14989, 21))


## Calculate filter cut-off frequency

In [202]:
fc_mult = 6.0
# this should be loaded from previously saved data
wf = 0.1
order = 6
fc = wf * order * fc_mult
print("cut frequency is {}".format(fc))

cut frequency is 3.6


## Get velocity and acceleration data and filter raw data

In [203]:
t_cut, q_f, dq_f, ddq_f, tau_f, q_raw_cut, tau_raw_cut = diff_and_filt_data(dof, 1.0/trajectory_sampling_rate, t,
                                                                        q_raw, dq_raw, tau_raw, fc, fc, fc, fc)
plot_trajectory_data(t_cut, q_raw_cut, q_f, dq_f, ddq_f, tau_raw_cut, tau_f)

q_raw shape: (14989, 7)
0
1
2
3
4
5
6
plot shape: 470


---
# Regression

## Ordinary Least Square (OLS)

### Generate regressor matrix for base parameters

In [204]:
base_param_num = robot_model.base_num
H_b_func = robot_model.H_b_func
W_b, tau_s = gen_regressor(base_param_num, H_b_func, q_f, dq_f, ddq_f, tau_f)

In [205]:
print(W_b.shape, tau_s.shape)
print np.linalg.cond(W_b)

((102123, 69), (102123,))
2800.124692934637


In [206]:
xb_ols = np.linalg.lstsq(W_b, tau_s)[0]

/home/yan/anaconda2/envs/dynamics/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [207]:
#sympy.Matrix.hstack(sympy.Matrix(robot_model.base_param), sympy.Matrix(xb_ols))

from IPython.display import HTML, display
import tabulate

table = [["Base Parameter", "Value"]]

for i in range(robot_model.base_num):
    param_str = str(sympy.Matrix(robot_model.base_param)[i])
    max_disp_len = 50
    line = [param_str if len(param_str) <= max_disp_len else param_str[:max_disp_len] + '...', xb_ols[i]]
    #['%.7s ...'%b if len(str(b)) > 7 else str(b)
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt='html')))

Base Parameter,Value
1.0*L7xy,-3.120585369137763e-05
1.0*L7xz,-9.95015975310265e-05
1.0*L7yz,-0.00010331558189266652
1.0*L6xy,-0.0007260047847384909
1.0*L6yz,3.8664931486461634e-05
1.0*L6xz,0.00016393523028139859
1.0*L6xx - 1.0*L6yy + 1.0*L7yy,-0.001403154320723684
1.0*L5xy,-5.7726489426692014e-05
1.0*L7zz,-0.0001498040720737514
1.0*l7y,0.0017086013418754079


## Compare measured torque and predicted torque

### Generate predicted torque

In [208]:
tau_p = np.zeros(tau_f.shape)
tau_ps = W_b.dot(xb_ols)
for i in range(dof):
    tau_p[:, i] = tau_ps[i::dof]
    

### Evaluate regression

In [209]:
var_regression_error_ols = np.linalg.norm(tau_ps - tau_s) / (tau_ps.size - base_param_num)
print("variance of regression error using OLS:")
print(var_regression_error_ols)

std_dev_xb_ols = np.sqrt(np.diag(var_regression_error_ols * np.linalg.inv(W_b.transpose().dot(W_b))))
print("standard deviation of xb using OLS:")
print(std_dev_xb_ols)

pct_std_dev_xb_ols = std_dev_xb_ols / np.abs(xb_ols)
print("percentage of standard deviation of xb using OLS: ")
print(pct_std_dev_xb_ols)

variance of regression error using OLS:
0.0005628336482166027
standard deviation of xb using OLS:
[6.36861025e-06 6.09278887e-06 5.38341034e-06 1.15075635e-05
 1.14780269e-05 7.62532308e-06 2.66163607e-05 2.37557767e-05
 9.07814378e-06 1.47966362e-05 1.44903900e-05 2.24409579e-05
 9.50244412e-06 3.17802073e-05 2.79934222e-05 1.21220956e-05
 3.56415516e-05 1.71678074e-05 3.24168365e-05 2.85435731e-05
 2.89076838e-05 3.35708509e-05 3.83531688e-05 2.91589161e-05
 6.95591621e-04 2.04010330e-03 2.71954938e-04 1.75361649e-04
 4.67435192e-05 9.47148249e-05 2.78877263e-04 2.49613475e-05
 3.41726020e-05 2.17237312e-04 3.38967005e-04 7.31259144e-05
 3.69918303e-05 8.75312646e-05 3.64981677e-05 4.22412515e-04
 2.22104520e-04 7.58447928e-04 8.56099864e-04 3.02775754e-04
 4.80685218e-04 1.46991480e-04 5.72382168e-04 6.60337549e-05
 1.89054902e-04 1.89486386e-04 4.16342041e-04 4.01584005e-04
 4.06643951e-03 4.75701670e-04 2.10055207e-04 1.98581393e-04
 1.99215051e-04 2.29128718e-04 1.96655884e-04 2.

### Plot measured torque and predicted torque

In [292]:
plot_data_num = int(1 / wf * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut[:plot_data_num], tau_f[:plot_data_num, :], tau_p[:plot_data_num, :])

## Weighted Least Square (WLS)

In [212]:
np.sqrt(np.linalg.norm(tau_f - tau_p, axis=0)/(tau_f.shape[0] - base_param_num))

array([0.03435196, 0.05475498, 0.04672867, 0.02332499, 0.02062491,
       0.01156464, 0.00841682])

In [282]:
weight = np.max(tau_f, axis=0) - np.min(tau_f, axis=0)
weight
weight[1] /= 1
weight[2] /= 1
weight[3] /= 1
weight

array([3.59432122, 5.61162704, 3.38184646, 0.50537823, 0.20274827,
       0.10040169, 0.0056625 ])

In [283]:
W_b.shape, tau_s.shape

In [284]:
# repeat the weight to generate a large vecoter for all the data
weights = 1.0/np.tile(weight, W_b.shape[0]/weight.shape[0])

In [285]:
weights.shape

In [286]:
W_b_wls = np.multiply(W_b, np.asmatrix(weights).transpose())
tau_s_wls = np.multiply(tau_s, weights)

In [287]:
xb_wls = np.linalg.lstsq(W_b_wls, tau_s_wls)[0]

/home/yan/anaconda2/envs/dynamics/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [288]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.base_param), sympy.Matrix(xb_wls))

⎡                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                   

In [289]:
tau_p_wls = np.zeros(tau_f.shape)
tau_ps_wls = W_b.dot(xb_wls)
for i in range(dof):
    tau_p_wls[:, i] = tau_ps_wls[i::dof]

In [294]:
plot_meas_pred_tau(t_cut[:plot_data_num], tau_f[:plot_data_num, :], tau_p_wls[:plot_data_num, :])

In [295]:
np.linalg.norm(tau_f[:plot_data_num, :] - tau_p_wls[:plot_data_num, :], axis=0)\
/ np.linalg.norm(tau_f[:plot_data_num, :], axis=0)

array([0.12096617, 0.40652311, 0.38659702, 0.40641481, 0.57916972,
       0.2873707 , 0.38780102])

## Convex optimization

### Generate regressor matrix for barycentric parameters

In [149]:
from identification import SDPOpt

bary_param_num = len(robot_model.bary_param)
H_func = robot_model.H_func
W, tau_s = gen_regressor(bary_param_num, H_func, q_f, dq_f, ddq_f, tau_f)

In [150]:
W_cvx = np.multiply(W, np.asmatrix(weights).transpose())
tau_s_cvx = np.multiply(tau_s, weights)

In [151]:
len(robot_model.std_param)

| min mass | max mass | min $r_x$ | max $r_x$ | min $r_y$ | max $r_y$ | min $r_z$ | max $r_z$ | max $F_c$ | max $F_v$ | max $F_o$ |
|----------|----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|--------------|-----------|

In [173]:
sdp_constraints = []
if model_name == 'mtm':
    sdp_constraints = [(0.5, 10, -0.1, 0.1, -0.1, 0.1, -0.1, 0.3, 0.2, 0.2, 0.2),
                      (0.3, 5, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 0.2, 0.2, 0.2),
                      (0.1, 3, -0.0, 0.3, -0.02, 0.02, -0.01, 0.01, 0.2, 0.2, 0.2),
                      (0.1, 1, -0.01, 0.01, -0.0, 0.1, -0.1, 0.0, 0.2, 0.2, 0.1),
                      (0.02, 0.3, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.2, -0.005, 0.005, -0.00, 0.05, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.01, 0.1, -0.000, 0.000, -0.00, 0.00, -0.00, 0.05, 0.1, 0.1, 0.1),
                      (0.3, 5, -0.10, 0.05, -0.05, 0.05, -0.05, 0.05, 0.1, 0.2, 0.2),
                      (0.05, 0.5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1),
                      (0.0, 0, 0.00, 0.00, -0.00, 0.00, 0.000, 0.00, 0.1, 0.1, 0.0)]
elif model_name == 'mtm_3links':
    sdp_constraints = [(0.5, 10, -0.1, 0.1, -0.1, 0.1, -0.1, 0.3, 0.5, 0.5, 0.5),
                      (0.3, 5, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 0.5, 0.5, 0.5),
                      (0.1, 3, -0.0, 0.3, -0.02, 0.02, -0.01, 0.01, 0.1, 0.5, 0.5),
                      (0.1, 1, -0.01, 0.01, -0.0, 0.1, -0.1, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.3, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.2, -0.005, 0.005, -0.00, 0.05, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.01, 0.1, -0.000, 0.000, -0.00, 0.00, -0.00, 0.05, 0.1, 0.1, 0.1),
                      (0.3, 5, -0.10, 0.05, -0.05, 0.05, -0.05, 0.05, 0.5, 0.5, 0.5),
                      (0.3, 5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1),
                      (0.3, 5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1)]
else:
    raise Exception("No robot name matches " + model_name)

In [174]:


sdp_opt_std = SDPOpt(W_cvx, tau_s_cvx, robot_model, sdp_constraints)
# sdp_opt_std = SDPOpt(W, tau_s, robot_model, sdp_constraints)
%time sdp_opt_std.solve()

Regressor shape: (102123, 120)
Regressand shape: (102123,)
Creating variables...
Creating optimization objective...
Creating constraints...
Solving problem...
----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 4268296, CG tol ~ 1/iter^(2.00)
eps = 1.00e-03, alpha = 1.50, max_iters = 10000, normalize = 1, scale = 1.00
Variables n = 121, constraints m = 102447
Cones:	linear vars: 133
	soc vars: 102125, soc blks: 1
	sd vars: 189, sd blks: 9
Setup time: 1.62e-01s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  3.50

  9400| 3.10e-01  1.06e-02  5.74e-04  1.25e+02  1.26e+02  5.87e-14  1.63e+03 
  9500| 3.10e-01  1.06e-02  5.65e-04  1.27e+02  1.27e+02  5.98e-14  1.65e+03 
  9600| 3.09e-01  1.00e-02  5.54e-04  1.28e+02  1.28e+02  6.08e-14  1.67e+03 
  9700| 3.09e-01  1.02e-02  5.43e-04  1.29e+02  1.29e+02  6.19e-14  1.69e+03 
  9800| 3.08e-01  1.05e-02  5.35e-04  1.30e+02  1.30e+02  6.30e-14  1.70e+03 
  9900| 3.08e-01  9.87e-03  5.24e-04  1.31e+02  1.32e+02  6.40e-14  1.72e+03 
 10000| 3.08e-01  9.73e-03  5.15e-04  1.33e+02  1.33e+02  6.51e-14  1.74e+03 
----------------------------------------------------------------------------
Status: Solved/Inaccurate
Hit max_iters, solution may be inaccurate
Timing: Solve time: 1.74e+03s
	Lin-sys: avg # CG iterations: 35.15, avg solve time: 1.72e-01s
	Cones: avg projection time: 2.59e-04s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 6.4829e-09, dist(y, K*) = 2.7388e-09, s'y/|s||y| = 2.0676e-11
|Ax + s -

## Compare measured torque and predicted torque

### Generate predicted torque

In [175]:
tau_sdp_p = np.zeros(tau_f.shape)
tau_sdp_ps = W.dot(sdp_opt_std.x_result)
for i in range(dof):
    tau_sdp_p[:, i] = tau_sdp_ps[i::dof]

### Plot measured torque and predicted torque

In [200]:
plot_data_num = int(1 / wf * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut[:plot_data_num] - t_cut[0],
                   tau_f[:plot_data_num, :],
                   tau_sdp_p[:plot_data_num, :])
# plot_meas_pred_tau(t_cut, tau_f, tau_sdp_p)

In [196]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.bary_param), sympy.Matrix(sdp_opt_std.x_result))

⎡L1xx    0.986192165056305  ⎤
⎢                           ⎥
⎢L1xy  -1.38516767370781e-14⎥
⎢                           ⎥
⎢L1xz  -7.76343556708024e-27⎥
⎢                           ⎥
⎢L1yy    0.986192165056345  ⎥
⎢                           ⎥
⎢L1yz  -2.12133368993413e-16⎥
⎢                           ⎥
⎢L1zz   5.58233934249789e-5 ⎥
⎢                           ⎥
⎢l1x   2.40588966703881e-27 ⎥
⎢                           ⎥
⎢l1y   4.76358117877349e-17 ⎥
⎢                           ⎥
⎢l1z     0.144175828054554  ⎥
⎢                           ⎥
⎢ m₁     1.79307211904898   ⎥
⎢                           ⎥
⎢Fc₁    0.0419135562259835  ⎥
⎢                           ⎥
⎢Fv₁    0.0497210098280573  ⎥
⎢                           ⎥
⎢Fo₁    -0.0698865408680767 ⎥
⎢                           ⎥
⎢L2xx   0.0194654973664062  ⎥
⎢                           ⎥
⎢L2xy   0.00876972571924054 ⎥
⎢                           ⎥
⎢L2xz   0.00593865075240333 ⎥
⎢                           ⎥
⎢L2yy   0.0330003696264969  ⎥
⎢         

In [178]:
from identification import barycentric2standard_params

In [179]:
sdp_opt_std.x_result

array([ 9.86192165e-01, -1.38516767e-14, -7.76343557e-27,  9.86192165e-01,
       -2.12133369e-16,  5.58233934e-05,  2.40588967e-27,  4.76358118e-17,
        1.44175828e-01,  1.79307212e+00,  4.19135562e-02,  4.97210098e-02,
       -6.98865409e-02,  1.94654974e-02,  8.76972572e-03,  5.93865075e-03,
        3.30003696e-02,  9.67284032e-04,  5.04838084e-02, -4.02232774e-01,
        6.80708808e-02,  6.43105023e-04,  5.00027870e+00,  1.09124988e-01,
        1.99956998e-01, -2.00003749e-01,  6.19401340e-04,  8.10870769e-04,
        6.31621227e-05,  1.18055051e-03, -3.76610295e-05,  1.71030714e-03,
        4.07898714e-02, -2.79045453e-02, -2.19072695e-03,  1.41404147e+00,
        1.00110498e-03,  9.95052980e-04,  2.00008574e-01,  2.02184077e-02,
        3.45713217e-03,  1.34006236e-03,  1.24674614e-02,  2.77597880e-03,
        1.57090952e-03, -8.13941900e-04,  1.04485762e-02, -1.29595950e-02,
        1.33525052e-01,  4.08774176e-02,  1.22874520e-02, -1.04736499e-04,
        7.97410096e-04, -

In [180]:
x_std = barycentric2standard_params(sdp_opt_std.x_result, robot_model)

In [181]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.std_param), sympy.Matrix(x_std))

⎡I1xx    0.974599396882406  ⎤
⎢                           ⎥
⎢I1xy  -1.38516767370781e-14⎥
⎢                           ⎥
⎢I1xz  -7.56998482337162e-27⎥
⎢                           ⎥
⎢I1yy    0.974599396882446  ⎥
⎢                           ⎥
⎢I1yz  -2.08303108884751e-16⎥
⎢                           ⎥
⎢I1zz   5.58233934249789e-5 ⎥
⎢                           ⎥
⎢r1x   1.34176960395484e-27 ⎥
⎢                           ⎥
⎢r1y   2.65665899779873e-17 ⎥
⎢                           ⎥
⎢r1z    0.0804071551405432  ⎥
⎢                           ⎥
⎢ m₁     1.79307211904898   ⎥
⎢                           ⎥
⎢Fc₁    0.0419135562259835  ⎥
⎢                           ⎥
⎢Fv₁    0.0497210098280573  ⎥
⎢                           ⎥
⎢Fo₁    -0.0698865408680767 ⎥
⎢                           ⎥
⎢I2xx   0.0185387373447571  ⎥
⎢                           ⎥
⎢I2xy   0.00329396309376292 ⎥
⎢                           ⎥
⎢I2xz   0.00588691805246235 ⎥
⎢                           ⎥
⎢I2yy  0.000643849538560864 ⎥
⎢         

In [182]:
sdp_constraints